In [2]:
import datetime
import mysql.connector
import pandas as pd
import os

# PHASE 1 : Création de la base et des tables
print("Connexion à MySQL pour création des tables...")
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='MichelSardou'
)
cursor = conn.cursor()

# Création de la base de données
print("Création de la base de données reddit_db...")
cursor.execute("DROP DATABASE IF EXISTS reddit_db")
cursor.execute("CREATE DATABASE reddit_db")
cursor.execute("USE reddit_db")

# Création des tables
print("Création des tables...")

tables_creation = [
    """
    CREATE TABLE AUTHOR (
        author VARCHAR(255) PRIMARY KEY
    )
    """,
    """
    CREATE TABLE SUBREDDIT (
        Subreddit_id VARCHAR(255) PRIMARY KEY,
        subreddit VARCHAR(255)
    )
    """,
    """
    CREATE TABLE SCORE (
        id VARCHAR(255) PRIMARY KEY,
        score INT,
        ups INT,
        downs INT,
        score_hidden BOOLEAN,
        gilded INT
    )
    """,
    """
    CREATE TABLE PARENT (
        parent_id VARCHAR(255) PRIMARY KEY,
        link_id VARCHAR(255)
    )
    """,
    """
    CREATE TABLE REMOVAL (
        removal_reason VARCHAR(255) PRIMARY KEY
    )
    """,
    """
    CREATE TABLE CONTROVERSY (
        controversiality INT PRIMARY KEY
    )
    """,
    """
    CREATE TABLE DISTINGUISHED (
        distinguished VARCHAR(255) PRIMARY KEY
    )
    """,
    """
    CREATE TABLE COMMENT (
        Id VARCHAR(255) PRIMARY KEY,
        created_utc DATETIME,
        name VARCHAR(255),
        body TEXT,
        edited BOOLEAN,
        author_flair_css_class VARCHAR(255),
        author_flair_text VARCHAR(255),
        author VARCHAR(255),
        subreddit_id VARCHAR(255),
        controversiality INT,
        FOREIGN KEY (author) REFERENCES AUTHOR(author),
        FOREIGN KEY (subreddit_id) REFERENCES SUBREDDIT(Subreddit_id),
        FOREIGN KEY (controversiality) REFERENCES CONTROVERSY(controversiality)
    )
    """,
    """
    CREATE TABLE AUTHORED (
        author VARCHAR(255),
        Id VARCHAR(255),
        PRIMARY KEY (author, Id),
        FOREIGN KEY (author) REFERENCES AUTHOR(author),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id)
    )
    """,
    """
    CREATE TABLE IS_IN (
        Id VARCHAR(255),
        Subreddit_id VARCHAR(255),
        PRIMARY KEY (Id, Subreddit_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (Subreddit_id) REFERENCES SUBREDDIT(Subreddit_id)
    )
    """,
    """
    CREATE TABLE DEPENDS (
        Id VARCHAR(255),
        parent_id VARCHAR(255),
        PRIMARY KEY (Id, parent_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (parent_id) REFERENCES PARENT(parent_id)
    )
    """,
    """
    CREATE TABLE HAS_SCORE (
        Id VARCHAR(255),
        score_id VARCHAR(255),
        PRIMARY KEY (Id, score_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (score_id) REFERENCES SCORE(id)
    )
    """,
    """
    CREATE TABLE REMOVED (
        Id VARCHAR(255),
        removal_reason VARCHAR(255),
        PRIMARY KEY (Id, removal_reason),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (removal_reason) REFERENCES REMOVAL(removal_reason)
    )
    """,
    """
    CREATE TABLE IS_CONTROVERSIAL (
        Id VARCHAR(255),
        controversiality INT,
        PRIMARY KEY (Id, controversiality),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (controversiality) REFERENCES CONTROVERSY(controversiality)
    )
    """,
    """
    CREATE TABLE IS_DISTINGUISHED (
        Id VARCHAR(255),
        distinguished VARCHAR(255),
        PRIMARY KEY (Id, distinguished),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (distinguished) REFERENCES DISTINGUISHED(distinguished)
    )
    """
]

for query in tables_creation:
    cursor.execute(query)

conn.commit()
print("Tables créées et commit envoyé.\n")
cursor.close()
conn.close()
print("Connexion fermée après la création des tables.\n")

# PHASE 2 : Réouverture pour l'import des CSV
print("Réouverture de la connexion pour l'import des données...")
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='MichelSardou',
    database='reddit_db'
)
cursor = conn.cursor()

print("Début de l'importation des CSV...")
folder = "data/excel"
csv_files = {
    'AUTHOR': 'askreddit_author.csv',
    'CONTROVERSY': 'askreddit_controverse.csv',
    'DISTINGUISHED': 'askreddit_distinguihshed.csv',
    'PARENT': 'askreddit_parent.csv',
    'REMOVAL': 'askreddit_removal.csv',
    'SCORE': 'askreddit_score.csv',
    'SUBREDDIT': 'askreddit_subreddit.csv',
    'COMMENT': 'askreddit_comment.csv',
    'DEPENDS': 'askreddit_depends.csv',
    'IS_DISTINGUISHED': 'askreddit_is_distinguihshed.csv',
    'REMOVED': 'askreddit_removed.csv'
}

for table, file in csv_files.items():
    print(f"\nImportation du fichier {file} dans la table {table}...")
    
    df = pd.read_csv(os.path.join(folder, file), dtype=str)
    df = df.where(pd.notnull(df), None)
    df.columns = df.columns.map(lambda x: str(x).strip())
    df = df.loc[:, df.columns.notna()]
    df = df.loc[:, df.columns != 'nan']

    # Conversion UTC
    if 'created_utc' in df.columns:
        df['created_utc'] = df['created_utc'].apply(
            lambda x: datetime.datetime.utcfromtimestamp(int(x)) if x else None
        )

    if 'edited' in df.columns:
        df['edited'] = df['edited'].apply(lambda x: 0 if (x == 'false' or x == '0' or x is None) else 1)

    if 'distinguished' in df.columns:
        df = df[df['distinguished'].notnull() & (df['distinguished'] != '')]

    clean_cols = list(df.columns)
    print(f"Colonnes utilisées pour {table} : {clean_cols}")

    cols = ",".join(clean_cols)
    placeholders = ",".join(["%s"] * len(clean_cols))
    insert_stmt = f"INSERT INTO {table} ({cols}) VALUES ({placeholders})"

    for row in df[clean_cols].itertuples(index=False):
        try:
            cursor.execute(insert_stmt, tuple(row))
        except Exception as e:
            print(f"Erreur sur {table} : {e}")

# Commit après toutes les insertions
conn.commit()
print("\nToutes les données ont été insérées et validées.")

cursor.close()
conn.close()
print("Connexion fermée après import des données.")

Connexion à MySQL pour création des tables...
Création de la base de données reddit_db...
Création des tables...
Tables créées et commit envoyé.

Connexion fermée après la création des tables.

Réouverture de la connexion pour l'import des données...
Début de l'importation des CSV...

Importation du fichier askreddit_author.csv dans la table AUTHOR...
Colonnes utilisées pour AUTHOR : ['author']

Importation du fichier askreddit_controverse.csv dans la table CONTROVERSY...
Colonnes utilisées pour CONTROVERSY : ['controversiality']

Importation du fichier askreddit_distinguihshed.csv dans la table DISTINGUISHED...
Colonnes utilisées pour DISTINGUISHED : ['distinguished']

Importation du fichier askreddit_parent.csv dans la table PARENT...
Colonnes utilisées pour PARENT : ['parent_id', 'link_id']

Importation du fichier askreddit_removal.csv dans la table REMOVAL...
Colonnes utilisées pour REMOVAL : ['removal_reason']

Importation du fichier askreddit_score.csv dans la table SCORE...
Colo